In [1]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import models
import encoders
import decoders
from training import train, test, checkpoint, main_wandb, main_simple
from dataloaders import load_data, make_loaders, append_SOS
import utils

import sys
sys.path.insert(0, '/scratch/users/udemir15/ELEC491/bassline_transcription')
from utilities import *

from bassline_transcriber.transcription import NN_output_to_MIDI_file

SEED = 27

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

wandb: Currently logged in as: raraz15 (use `wandb login --relogin` to force relogin)


cuda


In [10]:
M = 8

data_params = {'dataset_path': '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]/sus',
               'dataset_name': 'TechHouse_bassline_representations_4020',
               'scale_type': 'min',
               'M': M}

X, titles = load_data(data_params, True)

K = int(X.max()+1) # Number of classes, assumes consecutive [0,max] inclusive
sequence_length = X.shape[1]

print('\nNumber of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

FileNotFoundError: [Errno 2] File /scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]/sus/TechHouse_bassline_representations_4020_augmented_min_M8.csv does not exist: '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]/sus/TechHouse_bassline_representations_4020_augmented_min_M8.csv'

In [11]:
params = {
    "data": {
        "dataset_path": "/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]",
        "dataset_name": "TechHouse_bassline_representations_4020_augmented",
        "scale_type": "min",
        "M": 8
    },
    "encoder": {
        "embedding_size": 12,
        "hidden_size": 256,
        "output_size": 64
    },
    "training": {
        "batch_size": 128,
        "N_epochs": 2000,
        "lr": 0.0005,
        "teacher_forcing_ratio": 0.75
    }
}

In [12]:
encoder_params = params['encoder']
encoder_params['input_size'] = 27
decoder_params = {'output_size': 27,
              'embedding_size': encoder_params['embedding_size'],
              'hidden_size': encoder_params['output_size']}

In [16]:
state_dict = load_state_dict(model_name)

In [8]:
model_name = '08_06__21_03'
model_path = os.path.join('model_checkpoints', model_name+'.pt')

In [14]:
encoder = encoders.BidirectionalGRUEncoder(**encoder_params).to(device)
decoder = decoders.GRUDecoderWithAttention(**decoder_params).to(device)

model = models.Seq2SeqGRUWithAttention(encoder, decoder, device)

In [18]:
model.load_state_dict(state_dict)
model.eval()

Seq2SeqGRUWithAttention(
  (encoder): BidirectionalGRUEncoder(
    (embedding): Embedding(27, 12)
    (rnn): GRU(12, 256, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=512, out_features=64, bias=True)
  )
  (decoder): GRUDecoderWithAttention(
    (attention): Attention(
      (attn): Linear(in_features=91, out_features=64, bias=True)
      (v): Linear(in_features=64, out_features=1, bias=False)
    )
    (embedding): Embedding(27, 12)
    (rnn): GRU(39, 64, batch_first=True)
    (fc_out): Linear(in_features=103, out_features=27, bias=True)
  )
)

In [22]:
samples = model.sample(14)

In [23]:
samples

tensor([[14, 26, 14, 14, 14, 16,  1, 14,  1, 14, 14, 14, 14, 14,  1, 14, 14, 26,
         14, 16, 14, 14,  1, 14,  1, 14, 14, 14, 14, 14, 26, 14, 14,  1, 16, 14,
         14, 14,  1, 14, 26, 14, 14, 14, 14, 14,  1, 14, 14,  1, 16, 14, 26, 14,
          1, 14, 16, 14, 14, 14, 14, 14,  1, 14],
        [14, 26, 14, 14, 14, 16,  1, 14,  1, 14, 14, 14, 14, 14,  1, 14, 14, 26,
         14, 16, 14, 14,  1, 14,  1, 14, 14, 14, 14, 14, 26, 14, 14,  1, 16, 14,
         14, 14,  1, 14, 26, 14, 14, 14, 14, 14,  1, 14, 14,  1, 16, 14, 26, 14,
          1, 14, 16, 14, 14, 14, 14, 14,  1, 14],
        [14, 26, 14, 14, 14, 16,  1, 14,  1, 14, 14, 14, 14, 14,  1, 14, 14, 26,
         14, 16, 14, 14,  1, 14,  1, 14, 14, 14, 14, 14, 26, 14, 14,  1, 16, 14,
         14, 14,  1, 14, 26, 14, 14, 14, 14, 14,  1, 14, 14,  1, 16, 14, 26, 14,
          1, 14, 16, 14, 14, 14, 14, 14,  1, 14],
        [14, 26, 14, 14, 14, 16,  1, 14,  1, 14, 14, 14, 14, 14,  1, 14, 14, 26,
         14, 16, 14, 14,  1, 14,  1, 14,

In [25]:
#min_note = 28
#sustain_code = None
for i, sample in enumerate(samples.cpu().numpy()):
    NN_output_to_MIDI_file(sample, '{}'.format(i), 'midi/seq2seq_att', M)
                           #min_note=min_note,
                           #sustain_code=sustain_code)